In [1]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

C:\Users\UTEA\anaconda3\envs\utea_estimativas\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [10]:
import tkinter as tk
from tkinter import filedialog as fd

import pandas as pd

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(ruta + '/_keys/agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(ruta + '/_keys/llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [3]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [11]:
# open file dialog
root = tk.Tk()
root.attributes('-topmost', True)
root.iconify()
file_name = fd.askopenfilename(parent=root, filetypes=[(".xlsx", ".xlsx")])
root.destroy()

In [13]:
file_name

'D:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2023/ANALISIS DE CAÑA EN CAMPO/GRUPO COSECHA 2023.xlsx'

In [15]:
data = pd.read_excel(file_name)

In [18]:
data

,CODIGO CAÑERO,INST,NOMBRE CAÑERO,COD COSECHA,DELEGADO
0,17,131,ACSAMA SEGOVIA SABINO,356,0
1,18,126,ACUNA ARISPE HONORATO,354,0
2,39,59,AGUILAR ROMERO NICOLAS,32,0
3,47,66,AGUILERA ANEZ RAFAEL,238,0
4,62,50,AGUILERA PALMA JORGE ENRIQUE,418,0
...,...,...,...,...,...
1365,42262,131,JALDIN GARCIA ANDREA,359,0
1366,42263,131,JALDIN GARCIA DIEGO,359,0
1367,42264,131,MERCADO JALDIN MARCOS,359,0
1368,42265,131,MOLINA MONTAÑO DARWIN,362,0


In [6]:
cantidad_usuarios = dbQuery('select count(*) cantidad_usuarios from users where estado=\'activo\' and tipo = \'propietario\'')[0]['cantidad_usuarios']

In [5]:
cantidad_usuarios

53